# Challenge 3 - Avanzado

## Visualización de grandes bases de datos

### Eyder Uriel Kinil Cervera 

Este challenge avanzado permitirá a los estudiantes aplicar PySpark para analizar grandes volúmenes de datos relacionados con un tema cercano a su investigación de tesis, integrando los conocimientos adquiridos en los challenges anteriores.

### Objetivo general

Realizar un análisis avanzado utilizando PySpark, relacionando el proyecto con su tema de tesis.

### Objetivo especifico

Utilizar APIs avanzadas como Spark Streaming o Spark ML para trabajar con una gran base de datos en tiempo real o construir un modelo predictivo complejo. Los estudiantes deberán crear una presentación de su análisis y presentarla al grupo.